In [65]:
import sys
import numpy as np
import itertools
import matplotlib.pyplot as plt

In [2]:
sys.path.append('C:\Repos\WhoCell\who_cell')
sys.path.append('C:\Repos\WhoCell')
sys.path.append('C:\Repos\WhoCell\who_cell\experiments')
sys.path.append('C:/Repos/WhoCell/who_cell/models')
sys.path.append('C:\Repos\pomegranate')

from pomegranate import *


# build test network

In [6]:
from who_cell.simulation.simulation_for_gibbs import Simulator_for_Gibbs

In [59]:
n_states = 10
mutual_model_params_dict = {}
mutual_model_params_dict['N'] = 10
mutual_model_params_dict['d'] = 5
mutual_model_params_dict['n_states'] = n_states
max_number_of_sampled_traj = 500
is_acyclic = True


combined_params = {}
combined_params['p_prob_of_observation'] = 1
combined_params['number_of_smapled_traj'] = 500
combined_params['N'] = mutual_model_params_dict['N']
  

In [60]:

simulator = Simulator_for_Gibbs(mutual_model_params_dict['N'], mutual_model_params_dict['d'],
                                        mutual_model_params_dict['n_states'], easy_mode=True,
                                        max_number_of_sampled_traj =max_number_of_sampled_traj ) # we need max_number_of_sampled_traj to know how much traj to pre sample so the traj will be mutual

pome_results = simulator.build_pome_model(mutual_model_params_dict['N'], mutual_model_params_dict['d'],
                               simulator.mues, simulator.sigmas,is_acyclic)

(all_relvent_sampled_trajs_states),_ = simulator.simulate_observations(pome_results["model"],combined_params,pome_results['params_signature'],from_pre_sampled_traj = True)


In [61]:
all_relvent_traj = all_relvent_sampled_trajs_states[0]
all_obs_flat = list(itertools.chain(*all_relvent_traj))

# gmm

In [69]:
all_obs_flat_arr = np.array(all_obs_flat).reshape(len(all_obs_flat),1)
gmm_model = gmm.GeneralMixtureModel.from_samples(distributions  = NormalDistribution,
                                                 n_components  = n_states,
                                                 X = all_obs_flat_arr,n_init = 5)

In [76]:
predictions = gmm_model.predict(all_obs_flat_arr)

In [84]:
state_to_params = {}
for k in range(n_states) : 
    y_k = all_obs_flat_arr[predictions == k ]
    _max_k = y_k.max()
    _min_k = y_k.min()
    
    chi = (_max_k+_min_k)/2
    kapa = 1/(_max_k - _min_k)**2
    
    state_to_params[k] = (chi,kapa)
    

In [85]:
state_to_params

{0: (3.031549327129866, 3.2730813530192973),
 1: (10.006445152867455, 4.724168329233665),
 2: (8.046615494392936, 7.5919936308574005),
 3: (0.9631421435936807, 3.1041005063305964),
 4: (5.5169965909952285, 0.17384853515357163),
 5: (9.022864859456458, 4.647499946481179),
 6: (7.02829525895432, 5.268683432819331),
 7: (2.0366613160868257, 4.058822284921063),
 8: (3.984111261691891, 6.472614180120248),
 9: (5.008020491848237, 4.335755899361196)}